# Image Classification with Caffe

This tutorial demonstrates the steps required to prepare and deploy a trained Caffe model for FPGA acceleration using Xilinx MLSuite:  
1. **Quantize the model** - The quantizer will generate scaling parameters for quantizing floats INT8. This is required, because FPGAs will take advantage of Fixed Point Precision, to achieve more parallelization at lower power. 
2. **Compile the Model** - In this step, the network Graph (prototxt) and the Weights (caffemodel) are compiled, the compiler 
3. **Subgraph Cutting** - In this step, the original graph is cut, and a custom FPGA accelerated python layer is inserted to be used for Inference. 
4. **Classification** - In this step, the caffe model and the prototxt from the previous step are run on the FPGA to perform inference on an input image.
  
For command line versions see: examples/caffe/  
  
## Prerequisite Files
1. **Model files** - This notebook requires that model files are located in  
  `$VAI_ALVEO_ROOT/examples/caffe/models/`
2. **Image files** - This notebook requires ilsvrc2012 image files are downloaded in  
  `$HOME/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/`
  
## Setup (Before Running Notebook)
```
conda activate vitis-ai-caffe
python -m ck pull repo:ck-env
python -m ck install package:imagenet-2012-val-min
python -m ck install package:imagenet-2012-aux
head -n 500 $HOME/CK-TOOLS/dataset-imagenet-ilsvrc2012-aux/val.txt > $HOME/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/val_map.txt
cd $VAI_ALVEO_ROOT/examples/caffe
python resize.py $HOME/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min 256 256
python getModels.py
source $VAI_ALVEO_ROOT/overlaybins/setup.sh
python replace_mluser.py --modelsdir models
```

### Step 1. Import required packages

In [ ]:
from __future__ import print_function

import os
import shutil
import subprocess

from IPython.display import Image as display
from ipywidgets import interact

import numpy as np
from caffe import Classifier, io
from caffe.proto import caffe_pb2
from caffe.draw import draw_net_to_file
from google.protobuf import text_format

# Environment Variables ("source overlaybins/setup.sh")
VAI_ALVEO_ROOT = os.getenv("VAI_ALVEO_ROOT",os.getcwd()+"/..")
XCLBIN = "/opt/xilinx/overlaybins/xdnnv3"

print("Running w/ VAI_ALVEO_ROOT: %s" % VAI_ALVEO_ROOT)
print("Running w/ XCLBIN: %s" % XCLBIN)

# Bring in SubGraph Cutter
from decent import CaffeFrontend as xfdnnQuantizer
from vai.dpuv1.rt.scripts.framework.caffe.xfdnn_subgraph import CaffeCutter as xfdnnCutter

In [ ]:
# Delete stale directories
if os.path.exists("quantize_results"):
    shutil.rmtree("quantize_results")
if os.path.exists("work"):
    shutil.rmtree("work")

### Step 2. Choose a model
Choose a model using the drop down, or select custom, and enter your own.

In [ ]:
@interact(MODEL=["bvlc_googlenet","inception_v2","inception_v3","inception_v4",\
                    "resnet50_v1","resnet50_v2","squeezenet","vgg16","custom"])
def selectModel(MODEL):
    global prototxt
    global caffemodel
    global name
    model_root = VAI_ALVEO_ROOT + "/examples/caffe/models/"
    if MODEL == "custom":
        prototxt = None
        caffemodel = None
        name = None
    else:
        prototxt = model_root + MODEL + "/" + MODEL + "_train_val.prototxt"
        caffemodel = model_root + MODEL + "/" + MODEL + ".caffemodel"
        name = MODEL

In [ ]:
if not prototxt:
    @interact(PROTOTXT="Provide the path to your prototxt")
    def selectPrototxt(PROTOTXT):
        global prototxt
        prototxt = PROTOTXT
    @interact(CAFFEMODEL="Provide the path to your caffemodel")
    def selectCaffemodel(CAFFEMODEL):
        global caffemodel
        caffemodel = CAFFEMODEL
    @interact(MODEL="Provide a name to your model")
    def selectCaffemodel(MODEL):
        global name
        name = MODEL

In [ ]:
print("Currently running : %s" % name)
print("Running with prototxt:   %s" % prototxt)
print("Running with caffemodel: %s" % caffemodel)

### Step 3. Run the Quantizer

Here, we will quantize the model. The inputs are model prototxt, model weights, number of test iterations and calibration iterations. The output is quantized prototxt, weights, and quantize_info.txt and will be generated in the quantize_results/ directory.

The Quantizer will generate a json file holding scaling parameters for quantizing floats to INT8
This is required, because FPGAs will take advantage of Fixed Point Precision, to achieve accelerated inference

In [ ]:
def Quantize(prototxt,caffemodel,calib_iter=1,output_dir="quantize_results"):
    os.environ["DECENT_DEBUG"] = "1"
    subprocess.call(["vai_q_caffe", "quantize",
                 "--model", prototxt,
                 "--weights", caffemodel,
                 "--calib_iter", str(calib_iter)])


In [ ]:
Quantize(prototxt,caffemodel)

### Step 4: Run the Compiler

The compiler takes in the quantizer outputs from the previous step (prototxt, weights, quantize_info) and outputs a compiler.json and quantizer.json.

* A Network Graph (prototxt) and a Weights Blob (caffemodel) are compiled
* The network is optimized
* FPGA Instructions are generated

In [ ]:
arch = os.getenv("VAI_PYTHON_DIR")+"/vaic/arch/dpuv1/ALVEO/ALVEO.json" # Informs compiler what underlying hardware is capable of

def Compile(prototxt="quantize_results/deploy.prototxt",\
            caffemodel="quantize_results/deploy.caffemodel",\
            quantize_info="quantize_results/quantize_info.txt"):
    
    subprocess.call(["vai_c_caffe",
                    "--prototxt", prototxt,
                    "--caffemodel", caffemodel,
                    "--net_name", name,
                    "--output_dir", "work",
                    "--arch", arch,
                    "--options", "{\"quant_cfgfile\":\"%s\"}" %(quantize_info)])

In [ ]:
Compile()

### Step 4: Run the Subgraph Cutter

The subgraph cutter creates a custom python layer to be accelerated on the FPGA. The inputs are compiler.json, quantizer.json and model weights from the compiler step, as well as the FPGA xclbin. This outputs a cut prototxt file with FPGA references, to be used for inference. 

In [ ]:
def Cut(prototxt):
    
    cutter = xfdnnCutter(
        inproto="quantize_results/deploy.prototxt",
        trainproto=prototxt,
        outproto="xfdnn_auto_cut_deploy.prototxt",
        outtrainproto="xfdnn_auto_cut_train_val.prototxt",
        cutAfter="data",
        xclbin=XCLBIN,
        netcfg="work/compiler.json",
        quantizecfg="work/quantizer.json",
        weights="work/weights.h5"
    )
    
    cutter.cut()


In [ ]:
Cut(prototxt)

In [ ]:
# Lets visualize the new graph with the FPGA subgraph
net = caffe_pb2.NetParameter()
text_format.Merge(open("xfdnn_auto_cut_deploy.prototxt").read(), net)
draw_net_to_file(net,"xfdnn_auto_cut_deploy.png")
display("xfdnn_auto_cut_deploy.png")

### Step 5: Inference 

The inputs are the FPGA prototxt file, caffemodel weights, a test image, and the labels


In [ ]:
def Classify(prototxt,caffemodel,image,labels):
    classifier = Classifier(prototxt,caffemodel,
        mean=np.array([104,117,123]),
        raw_scale=255, channel_swap=[2,1,0])

    predictions = classifier.predict([io.load_image(image)]).flatten()
    labels = np.loadtxt(labels, str, delimiter='\t')
    top_k = predictions.argsort()[-1:-6:-1]
    for l,p in zip(labels[top_k],predictions[top_k]):
        print (l," : ",p)

In [ ]:
# Choose image to run, display it for reference
HOME = os.getenv("HOME")
image = HOME+"/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/ILSVRC2012_val_00000002.JPEG"
display(filename=image) 

In [ ]:
Classify("xfdnn_auto_cut_deploy.prototxt",caffemodel,image,HOME+"/CK-TOOLS/dataset-imagenet-ilsvrc2012-aux/synset_words.txt") 

# Conclusion
This notebook demonstrates how to target Xilinx FPGAs for inference using Caffe.    
When the time comes to take your application to production please look at examples in /workspace/alveo/examples/deployment_modes/  
Highest performance is acheived by creating multiprocess pipelines.